In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

LET'S TRY SOME BASIC ANOMALY DETECTION ON PUMP-SENSOR-DATA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

In [ ]:
from sklearn.cluster import KMeans

Asset Management can exploit Sensor Data and do some kool anomaly detection using machine learning techniques!!

In [ ]:
!ls

In [ ]:
df = pd.read_csv('/kaggle/input/pump-sensor-data/sensor.csv')
df.info()

2 quick observation 
1. sensor 15 is use less
2. we have to do something for timestamp data type

In [ ]:
del df['sensor_15']

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
### one more unnecessary column
del df['Unnamed: 0']

Now ~ next problem is gonna be missing value!! got to do something 

In [ ]:

df.nunique() ## many repeating value here

many redundant columns

In [ ]:
##df.hist(df.columns, bins=25, layout=(8,7), figsize=(20, 18))
##plt.show() 
## not much info here 

let's drop duplicate rows first if any

In [ ]:
df.shape

In [ ]:
df = df.drop_duplicates()
df.shape

this is common practice in missing value analyses -- top n column with highest missing values 

In [ ]:
def calc_percent_NAs(df):
    nans = pd.DataFrame(df.isnull().sum().sort_values(ascending=False)/len(df), columns=['percent']) 
    idx = nans['percent'] > 0
    return nans[idx]

calc_percent_NAs(df).head(10)

we are following very simple imputation strategy over here (there are many shopisticated ones)
mean imputation for all columns that have more than 1% missing value 
(also one can remove the entire column if it has let's say more than 60% missing value. The upper limit is more of heuristic and experience based.)

In [ ]:
#better code is required
df['sensor_50'].fillna((df['sensor_50'].mean()), inplace=True)
df['sensor_51'].fillna((df['sensor_51'].mean()), inplace=True)
df['sensor_00'].fillna((df['sensor_00'].mean()), inplace=True)
df['sensor_08'].fillna((df['sensor_08'].mean()), inplace=True)
df['sensor_07'].fillna((df['sensor_07'].mean()), inplace=True)
df['sensor_06'].fillna((df['sensor_06'].mean()), inplace=True)
df['sensor_09'].fillna((df['sensor_09'].mean()), inplace=True)

In [ ]:
calc_percent_NAs(df).head(10)

In [ ]:
df = df.dropna()

In [ ]:
df.info()

guess we are ready.
make df a time seires first. set date as index.

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])


In [ ]:
df = df.set_index('timestamp')

In [ ]:
df.head() ## our time series is ready!! congrats

DON'T FORGET THE OUTLIERS!! THEY ARE THE ONES WHO MIGHT HAVE PLAYED ROLE IN ANOMALY....
SIMPLY REMOVING THEM MIGHT NOT BE A VERY GOOD IDEA

LET'S HAVE A LOOK AT MACHINE STATUS FIRST

In [ ]:
df['machine_status'].unique()

In [ ]:

dfBroken = df[df['machine_status']=='BROKEN']
dfSensors = df.drop(['machine_status'], axis=1)
sensorNames=dfSensors.columns
for sensor in sensorNames:
    sns.set_context('talk')
    _ = plt.figure(figsize=(18,3))
    _ = plt.plot(dfBroken[sensor], linestyle='none', marker='X', color='red', markersize=12)
    _ = plt.plot(df[sensor], color='grey')
    _ = plt.title(sensor)
    plt.show()

I WILL NOT CHECK THE STATIONARITY OF THIS LONG SIRIES. 
AGAIN FROM MY MULTIPLE YEARS IN TIME SERIES ALMOST ALL HIGH FREQUENCY LONG TIME SERIES WILL PASS THE STATIONARY TEST. YOU CAN CHECK THIS. FOR LONG SERIES OF HIGH FREQUENCY DATA ECONOMERTICIAN SHOULD BETTER COME WITH NEW TEST RATHER THAN RELYING UPON TRADITIONAL ONES WHICH WORKS PERFECTLY ON LOW FREQUENCY DATA WITH LESS RECORDS.

NOW SOME DIMENSION REDUCTION -- WILL GO AHEAD WITH PCA BUT SOON WRITE ANOTHER NOTEBOOK ON TICK (CLUSTERING). HAVE SEEN IT'S USE GETTING POPULAR IN INDUSTRY.

In [ ]:
x = df[sensorNames]
scaler = StandardScaler()
pca = PCA()
pipeline = make_pipeline(scaler, pca)
pipeline.fit(x)

In [ ]:
features = range(pca.n_components_)
_ = plt.figure(figsize=(22, 5))
_ = plt.bar(features, pca.explained_variance_)
_ = plt.xlabel('PCA')
_ = plt.ylabel('Var')
_ = plt.xticks(features)
_ = plt.title("Important PCs")
plt.show()

I WPN'T CHECK FOR AUTO-CORRELATION IN PCs (WHY? THEY ARE ORTHOGONAL)
WHAT ABOUT STATIONARITY AGAIN ADF WILL SAY THEY STATIONAY (CHECK IT IF YOU LIKE) 

In [ ]:
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = ['PC0', 'PC1','PC2'])

In [ ]:
df['PC0']=pd.Series(principalDf['PC0'].values, index=df.index)
df['PC1']=pd.Series(principalDf['PC1'].values, index=df.index)
df['PC2']=pd.Series(principalDf['PC2'].values, index=df.index)

IQR BEFORE ANYTHING

In [ ]:
#  for PC0
q1_pc1, q3_pc1 = df['PC0'].quantile([0.25, 0.75])
iqr_pc1 = q3_pc1 - q1_pc1
lower_pc1 = q1_pc1 - (1.5*iqr_pc1)
upper_pc1 = q3_pc1 + (1.5*iqr_pc1)

#PC1
q1_pc2, q3_pc2 = df['PC1'].quantile([0.25, 0.75])
iqr_pc2 = q3_pc2 - q1_pc2
lower_pc2 = q1_pc2 - (1.5*iqr_pc2)
upper_pc2 = q3_pc2 + (1.5*iqr_pc2)
#PC2
q1_pc3, q3_pc3 = df['PC2'].quantile([0.25, 0.75])
iqr_pc3 = q3_pc3 - q1_pc3
lower_pc3 = q1_pc3 - (1.5*iqr_pc3)
upper_pc3 = q3_pc3 + (1.5*iqr_pc3)

In [ ]:
print(lower_pc1, upper_pc1)
print(lower_pc2, upper_pc2)
print(lower_pc3, upper_pc3)

In [ ]:
df['anomaly_pc1'] = ((df['PC0']>upper_pc1) | (df['PC0']<lower_pc1)).astype('int')
df['anomaly_pc2'] = ((df['PC1']>upper_pc2) | (df['PC1']<lower_pc2)).astype('int')
df['anomaly_pc3'] = ((df['PC2']>upper_pc3) | (df['PC2']<lower_pc3)).astype('int')

In [ ]:
print(df['anomaly_pc1'].value_counts())
print(df['anomaly_pc2'].value_counts())
print(df['anomaly_pc3'].value_counts())

In [ ]:
outliers_pc1 = df.loc[(df['PC0']>upper_pc1) | (df['PC0']<lower_pc1), 'PC0']
outliers_pc2 = df.loc[(df['PC1']>upper_pc2) | (df['PC1']<lower_pc2), 'PC1']
outliers_pc3 = df.loc[(df['PC2']>upper_pc3) | (df['PC2']<lower_pc3), 'PC2']

In [ ]:
print(len(outliers_pc1)/len(df))
print(len(outliers_pc2)/len(df))
print(len(outliers_pc3)/len(df))

let's see what appears to be best three features if machine status is our variable of intrest

In [ ]:
# Apply SelectKBest class to extract the best 3 features - Univariate feature selection 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
x = df.drop(['machine_status', 'PC0','PC1' ,'PC2', 'anomaly_pc1', 'anomaly_pc2', 'anomaly_pc3'], axis=1)
y = df['machine_status']
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
bestfeatures = SelectKBest(score_func=chi2, k=3)
fit = bestfeatures.fit(x_scaled, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)
featureScores = pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns = ['Feature', 'Score']
print(featureScores.nlargest(3, 'Score'))

LET'S TRY FIGURE HOW PCs PERFORMED 

MANY SUCH COMBOS MIGHT BRING OUT BETTER INSIGHTS

In [ ]:

#PC3 ANOMALY on sensor_11 AND WHEN ACTUAL BREAK HAPPENED
a = df[df['anomaly_pc1'] == 1] #anomaly
b = df[df['anomaly_pc2'] == 1]
c = df[df['anomaly_pc3'] == 1]
_ = plt.figure(figsize=(18,6))
_ = plt.plot(df['sensor_11'], color='grey', label='Normal')
_ = plt.plot(a['sensor_11'], linestyle='none', marker='X', color='blue', markersize=12, label='pc1 Anomaly', alpha = 0.2)
_ = plt.plot(b['sensor_11'], linestyle='none', marker='X', color='green', markersize=12, label='pc2 Anomaly')
_ = plt.plot(c['sensor_11'], linestyle='none', marker='X', color='yellow', markersize=12, label='pc3 Anomaly', alpha=0.2)
_ = plt.plot(dfBroken['sensor_11'], linestyle='none', marker='X', color='red', markersize=12, label='Broken')
_ = plt.xlabel('Date and Time')
_ = plt.ylabel('Sensor Reading')
_ = plt.title('Sensor_11 Anomalies')
_ = plt.legend(loc='best')
plt.show();

of course PC2 and PC3 DOMINATES on sensor 11

NOW LET'S TRY K-MEANS

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(principalDf.values)
labels = kmeans.predict(principalDf.values)
unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))

In [ ]:

_ = plt.figure(figsize = (9, 7))
_ = plt.bar(clusters[0], clusters[1], tick_label=clusters[0])
_ = plt.xlabel('Clusters ON PCA')
_ = plt.ylabel('Number of RECORDS')
_ = plt.title('Number of RECORDS in cluster')
plt.show()

In [ ]:
principalDf.info()

In [ ]:
_ = plt.figure(figsize=(9,7))
_ = plt.scatter(principalDf['PC0'], principalDf['PC1'], c=labels)
_ = plt.xlabel('PC0')
_ = plt.ylabel('PC1')
_ = plt.title('K-means of clustering')
plt.show()

In [ ]:
_ = plt.figure(figsize=(9,7))
_ = plt.scatter(principalDf['PC1'], principalDf['PC2'], c=labels)
_ = plt.xlabel('PC1')
_ = plt.ylabel('PC2')
_ = plt.title('K-means of clustering')
plt.show()

In [ ]:
_ = plt.figure(figsize=(9,7))
_ = plt.scatter(principalDf['PC0'], principalDf['PC2'], c=labels)
_ = plt.xlabel('PC0')
_ = plt.ylabel('PC2')
_ = plt.title('K-means of clustering')
plt.show()

In [ ]:

from mpl_toolkits import mplot3d

In [ ]:
# Creating figure
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
 
# Creating plot
ax.scatter3D(principalDf['PC0'], principalDf['PC1'], principalDf['PC2'], c=labels)
plt.title("Anomaly Clusters ON PCA")
 
# show plot
plt.show()

3D plots looks ok!! we can create k=2 clusters as well!! for broken or not-broken 

LETS CHECK DISTANCE FROM NEAREST CENTROID

In [ ]:

def getDistanceByPoint(data, model):
    distance = []
    for i in range(0,len(data)):
        Xa = np.array(data.loc[i])
        Xb = model.cluster_centers_[model.labels_[i]]
        distance.append(np.linalg.norm(Xa-Xb))
    return pd.Series(distance, index=data.index)

LET'S FIND A THRESHOLD DISTANCE

In [ ]:
distance = getDistanceByPoint(principalDf, kmeans)

In [ ]:
outliers_fraction = 0.11
number_of_outliers = int(outliers_fraction*len(distance))

In [ ]:

threshold = distance.nlargest(number_of_outliers).min()

In [ ]:
#(0:normal, 1:anomaly) 
principalDf['anomalyDist'] = (distance >= threshold).astype(int)

In [ ]:
principalDf['anomalyDist'].value_counts()

In [ ]:
df['anomalyDist'] = pd.Series(principalDf['anomalyDist'].values, index=df.index)

In [ ]:
a = df[df['anomalyDist'] == 1] #anomaly
_ = plt.figure(figsize=(18,6))
_ = plt.plot(df['sensor_04'], color='grey', label='Normal')
_ = plt.plot(a['sensor_04'], linestyle='none', marker='X', color='blue', markersize=12, label='Dist Anomaly', alpha= 0.2)
_ = plt.plot(dfBroken['sensor_04'], linestyle='none', marker='X', color='red', markersize=12, label='Broken')
_ = plt.xlabel('Date and Time')
_ = plt.ylabel('Sensor Reading')
_ = plt.title('Centroid Anomalies on Sensor_04 ')
_ = plt.legend(loc='best')
plt.show();

ABOVE PICS SHOW DISTANCE BASED ANOMALIES MISSED A LOT OF BROKEN MACHINE STATUS 

In [ ]:
df[df['anomalyDist']==1]['machine_status'].value_counts()

In [ ]:
df['machine_status'].value_counts()

COULD CAPTURE 1 OF 7

LET'S TAKE HELP OF SCI-KIT

In [ ]:
from sklearn.ensemble import IsolationForest
outliers_fraction = 0.11 ## why
model =  IsolationForest(contamination=outliers_fraction)
model.fit(principalDf.values) 
principalDf['anomalyForest'] = pd.Series(model.predict(principalDf.values)) ### here -1 is anomaly

In [ ]:
df['anomalyForest'] = pd.Series(principalDf['anomalyForest'].values, index=df.index)
a = df.loc[df['anomalyForest'] == -1] #anomaly
_ = plt.figure(figsize=(18,6))
_ = plt.plot(df['sensor_04'], color='grey', label='Normal')
_ = plt.plot(a['sensor_04'], linestyle='none', marker='X', color='blue', markersize=12, label='Forest Anomaly', alpha=0.3)
_ = plt.plot(dfBroken['sensor_04'], linestyle='none', marker='X', color='red', markersize=12, label='Broken')
_ = plt.xlabel('Date and Time')
_ = plt.ylabel('Sensor Reading')
_ = plt.title('Sensor_04 FOREST ISOLATION Anomalies')
_ = plt.legend(loc='best')
plt.show();

COMPARE THE HITS AND MISSES OF LAST TWO GRAPHS!! SCI-KIT FOREST ISOLATION WINS....

INTERM OF NUMBERS 

In [ ]:
df['anomalyForest'].value_counts()

In [ ]:
df[df['anomalyForest']==1]['machine_status'].value_counts()

In [ ]:
df['machine_status'].value_counts()

6 of 7 TIMES| NOT BAD FOREST ISOLATION ||